Import packages

In [1]:
import numpy as np
import pandas as pd
from cube.api import Cube
cube=Cube(verbose=True)
import re
import nltk
from nltk.corpus import stopwords
from load_baseline_crf import *

/Users/zaizaiiswandering/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("/Users/zaizaiiswandering/Desktop/学校/大学/大三/nlp/project/job/Jobs_NYC_Postings_20231126.csv")
df = df.loc[:20]
df = df.dropna(subset=['Minimum Qual Requirements', 'Preferred Skills'], how='all')

Stopwords Removal

In [3]:
target = df.loc[:, ['Job ID', 'Minimum Qual Requirements', 'Preferred Skills']]
target = target.fillna('nan')

In [4]:
dic = ['', '¢', 'â', '*']
def remove_special_character(text, dic):
    s = ''
    for word in text:
        if word not in dic:
            s += word
    return s

In [5]:
target['Minimum Qual Requirements'] = target['Minimum Qual Requirements'].apply(lambda x : remove_special_character(x, dic))
target['Preferred Skills'] = target['Preferred Skills'].apply(lambda x : remove_special_character(x, dic))
target['skills'] = target['Minimum Qual Requirements'] + target['Preferred Skills']
target = target.drop(columns = ['Minimum Qual Requirements', 'Preferred Skills'])
target

,Job ID,skills
0,606346,1. A masters degree from an accredited colleg...
1,571361,At least six years of full-time satisfactory e...
2,600579,1. A baccalaureate degree from an accredited c...
3,569683,1. A baccalaureate degree from an accredited c...
4,614260,A four-year high school diploma or its educati...
5,599185,Qualification Requirements A four-year high s...
6,607141,(1) A baccalaureate degree from an accredited ...
7,580478,Qualification Requirements 1. A baccalaureate...
8,572055,1. A baccalaureate degree from an accredited c...
9,552150,1. A four-year high school diploma or its educ...


Sentence Segmentation + boi

In [6]:
cube.load("en")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def boi(txt):
    try:
        txt = cube(txt)
        sentence_num = 1
        lemma, pos, sentence_number, tag = [], [], [], []
        for sentence in txt.sentences:
            for token in sentence.tokens:
                for word in token.words:
                    pos.append(word.xpos)
                    lemma.append(word.lemma)
                    sentence_number.append(sentence_num)
                    tag.append('?')
            sentence_num += 1
    except:
        sentence_num = 1
        lemma, pos, sentence_number, tag = [], [], [], []
        for s in nltk.sent_tokenize(txt):
            pos_tagged_sentences = nltk.pos_tag(nltk.word_tokenize(s))
            for item in pos_tagged_sentences:
                pos.append(item[1])
                lemma.append(item[0])
                sentence_number.append(sentence_num)
                tag.append('?')
        sentence_num += 1
                
    df_pos = pd.DataFrame({'sentence_id': sentence_number,
                           'word': lemma,
                           'pos': pos,
                           'tag': tag})

    X_test = prepare_prompt(df_pos)
    y_pred = crf.predict(X_test)
        
    boi_list = []
    for l in y_pred:
        boi_list += l
    
    df_pos['tag'] = boi_list
    # only keeps the columns where tag not equals to O
    
    b, s, result = False, '', []
    for row in df_pos.iterrows():
        if row[1]['tag'][0] == 'B':
            b = True
            if row[1]['word'] != '.<UNK>' and row[1]['word'] != '<UNK>':
                s += str(row[1]['word'])
                s += ' '
        elif b and row[1]['tag'][0] == 'I':
            if row[1]['word'] != '.<UNK>' and row[1]['word'] != '<UNK>':
                s += str(row[1]['word'])
                s += ' '
        else:
            if b:
                s = s[: -1]
                result.append(s)
            b = False
            s = ''

    return result

In [12]:
target['Extracted_Skills'] = target['skills'].apply(lambda x : boi(x))
target

,Job ID,skills,Extracted_Skills
0,606346,1. A masters degree from an accredited colleg...,"[master 's degree, professional, economics, fi..."
1,571361,At least six years of full-time satisfactory e...,[construction management work on capital proje...
2,600579,1. A baccalaureate degree from an accredited c...,"[baccalaureate degree, analytic and quantitati..."
3,569683,1. A baccalaureate degree from an accredited c...,"[baccalaureate degree, provide employment plan..."
4,614260,A four-year high school diploma or its educati...,"[Education, administrative support .office man..."
5,599185,Qualification Requirements A four-year high s...,"[education, satisfactory clerical experience, ..."
6,607141,(1) A baccalaureate degree from an accredited ...,"[baccalaureate degree, four ( 24 ) semester cr..."
7,580478,Qualification Requirements 1. A baccalaureate...,"[baccalaureate degree, accounting, eight ( 8 )..."
8,572055,1. A baccalaureate degree from an accredited c...,"[baccalaureate degree, provide employment plan..."
9,552150,1. A four-year high school diploma or its educ...,"[computer maintenance and repair, state 's Dep..."


In [13]:
target.to_csv('result.csv')